# **Catálogos**

**Descripción:**

 El objetivo es determinar si el conjunto actual de catálogos es suficiente o si se podrían crear catálogos adicionales para mejorar las capacidades de filtrado en reportes futuros.

**Autor:** ```Simón Correa Marín```

**Fecha:** ```05/07/2025```


#### **Librerías**

In [1]:
# Librerías más importantes para el proceso de análisis exploratorio de datos (EDA)
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

#### **Carga de los datos**

In [2]:
# Ruta del archivo que contiene las tablas
ruta = r"..\data\tablas_limpias_prueba.xlsx"

# Se cargan los datos desde el archivo Excel
hojas = pd.read_excel(ruta, sheet_name=None)

# Diccionario global para almacenar los DataFrames
dfs = globals() 

# Se muestran las primeras filas de cada DataFrame
for nombre, df in hojas.items():
    dfs[nombre] = df  # Se guarda cada DataFrame como variable global
    print(f"\n{nombre} shape: {df.shape}")
    display(df.head(2))


clientes shape: (42914, 8)


,id,cod_tip_doc,cod_genero,desc_genero,nombre,fecha_nacimiento,ingresos,cod_sucursal
0,1092347697,1,M,Masculino,Edgar Javier VERA VERA,1995-02-04,30589665,202
1,1007686547,1,M,Masculino,Jordan Steven Ramirez RODRIGUEZ ...,1986-07-17,26600681,184



cat_tip_doc shape: (10, 2)


,cod_tip_doc,desc_tip_doc
0,6,Id Extranjero PN
1,7,Id Extranjero PJ



cat_sucursal shape: (963, 4)


,cod_sucursal,desc_sucursal,cod_pais_region_zona,cod_ciudad
0,1,Buenavista Ii,1690423,169008001000
1,2,20 De Julio,1690610,169011001000



cat_ciudad shape: (9402, 4)


,cod_ciudad,desc_ciudad,desc_departamento,desc_pais
0,169000000000,Sin Informacion,Sin Informacion,Colombia
1,169005000000,Sin Informacion,Antioquia,Colombia



cat_zona shape: (89, 4)


,cod_pais_region_zona,desc_zona,desc_region,desc_pais
0,1690000,Sin Zona,Sin Región,Colombia
1,1690100,Sin Zona,Antioquia,Colombia


#### **Catálogos actuales**

- cat_tip_doc
- cat_sucursal
- cat_ciudad
- cat_zona

#### **Catálogos propuestos**

- cat_genero
- cat_departamento
- cat_pais
- cat_region

#### **cat_genero**

In [3]:
# Catálogo de género eliminando duplicados
cat_genero = clientes[['cod_genero', 'desc_genero']].drop_duplicates().reset_index(drop=True)

# Se verifica el catálogo creado
display(cat_genero)

,cod_genero,desc_genero
0,M,Masculino
1,F,Femenino
2,N,No informa


In [4]:
# Se elimina la columna de descripción
clientes.drop(columns=['desc_genero'], inplace=True)

# Se verifican cambios
print(clientes.columns)

Index(['id', 'cod_tip_doc', 'cod_genero', 'nombre', 'fecha_nacimiento',
       'ingresos', 'cod_sucursal'],
      dtype='object')


#### **Catálogos geográficos**

**Hallazgos**

El campo cod_cuidad tiene un formato de 12 dígitos ```PPPDDDMMMCCC```
- PPP → Código de país (ej: 169 para Colombia, 250 para Francia)
- DDD → Departamento (solo para Colombia y países con subdivisión)
- MMM → Municipio o unidad territorial
- CCC → Ciudad/localidad específica (puede ser 000)

In [5]:
# Rellenar cod_ciudad con ceros a la izquierda
cat_ciudad["cod_ciudad"] = cat_ciudad["cod_ciudad"].astype(str).str.zfill(12)

# Extraer códigos jerárquicos
cat_ciudad["cod_pais"] = cat_ciudad["cod_ciudad"].str[0:3]
cat_ciudad["cod_departamento"] = cat_ciudad["cod_ciudad"].str[3:6]
# Si se requiere en el futuro: Municipio = [6:9], Localidad = [9:12]

# Crear catálogos propuestos
cat_pais = cat_ciudad[['cod_pais', 'desc_pais']].drop_duplicates().reset_index(drop=True)
cat_departamento = cat_ciudad[['cod_pais', 'cod_departamento', 'desc_departamento']].drop_duplicates().reset_index(drop=True)

# Limpiar cat_ciudad dejando solo información necesaria
cat_ciudad = cat_ciudad[['cod_ciudad', 'desc_ciudad', 'cod_departamento', 'cod_pais']]

**Lo mismo sucede en cat_zona, de allí podemos obtener el cod_region**

```PPP RR ZZ```
- PPP (3 dígitos): Código del país
- RR (2 dígitos): Código de región (ej: Antioquia, Caribe, etc.)
- ZZ (2 dígitos): Código de zona específica dentro de la región


In [6]:
# Asegurar que el código tiene 7 dígitos
cat_zona["cod_pais_region_zona"] = cat_zona["cod_pais_region_zona"].astype(str).str.zfill(7)

# Extraer cod_pais y cod_region
cat_zona["cod_pais"] = cat_zona["cod_pais_region_zona"].str[0:3]
cat_zona["cod_region"] = cat_zona["cod_pais_region_zona"].str[3:5]
cat_zona["cod_zona"] = cat_zona["cod_pais_region_zona"].str[5:7]

# Crear catálogo de regiones
cat_region = cat_zona[["cod_region", "desc_region"]].drop_duplicates().reset_index(drop=True)

# Limpiar cat_zona dejando solo lo necesario
cat_zona = cat_zona[["cod_pais_region_zona", "desc_zona", "cod_region", "cod_pais"]]

In [7]:
# Maestra y catálogos
tablas = {
    "clientes": clientes,
    "cat_tip_doc": cat_tip_doc,
    "cat_sucursal": cat_sucursal,
    "cat_ciudad": cat_ciudad,
    "cat_zona": cat_zona,
    "cat_genero": cat_genero,
    "cat_pais": cat_pais,
    "cat_departamento": cat_departamento,
    "cat_region": cat_region
}

# Mostrar columnas por tabla
for nombre, df in tablas.items():
    print(f"\nColumnas de {nombre}:")
    print(list(df.columns))



Columnas de clientes:
['id', 'cod_tip_doc', 'cod_genero', 'nombre', 'fecha_nacimiento', 'ingresos', 'cod_sucursal']

Columnas de cat_tip_doc:
['cod_tip_doc', 'desc_tip_doc']

Columnas de cat_sucursal:
['cod_sucursal', 'desc_sucursal', 'cod_pais_region_zona', 'cod_ciudad']

Columnas de cat_ciudad:
['cod_ciudad', 'desc_ciudad', 'cod_departamento', 'cod_pais']

Columnas de cat_zona:
['cod_pais_region_zona', 'desc_zona', 'cod_region', 'cod_pais']

Columnas de cat_genero:
['cod_genero', 'desc_genero']

Columnas de cat_pais:
['cod_pais', 'desc_pais']

Columnas de cat_departamento:
['cod_pais', 'cod_departamento', 'desc_departamento']

Columnas de cat_region:
['cod_region', 'desc_region']


**Acciones tomadas:**
- Se creó el catálogo `cat_pais` a partir del código y nombre del país.
- Se creó el catálogo `cat_departamento` a partir del país y departamento.
- Se dejó `cat_ciudad` únicamente con las columnas `cod_ciudad`, `desc_ciudad`, `cod_departamento` y `cod_pais`.
- Se creó el catálogo `cat_region` con los campos `cod_region` y `desc_region`.
- Se eliminó la descripción redundante de país y región en la tabla `cat_zona`, manteniendo solo:
  - `cod_pais_region_zona`, `desc_zona`, `cod_region`, `cod_pais`

In [8]:
# Se exporta a un solo archivo Excel con hojas separadas
ruta_salida = "../data/tablas_finales.xlsx"

with pd.ExcelWriter(ruta_salida, engine='openpyxl') as writer:
    for nombre, df in tablas.items():
        df.to_excel(writer, sheet_name=nombre, index=False)

print(f"Archivo exportado exitosamente en: {ruta_salida}")

Archivo exportado exitosamente en: ../data/tablas_finales.xlsx


In [9]:
import os

# Ruta de salida
ruta_csv = "../data/csv/"
os.makedirs(ruta_csv, exist_ok=True)  # Crea la carpeta si no existe

# Se exporta cada tabla como archivo CSV
for nombre, df in tablas.items():
    ruta_archivo = os.path.join(ruta_csv, f"{nombre}.csv")
    df.to_csv(ruta_archivo, index=False, encoding='utf-8-sig')  # 'utf-8-sig' para Excel

print(f"Archivos CSV exportados exitosamente en: {ruta_csv}")

Archivos CSV exportados exitosamente en: ../data/csv/
